In [ ]:
Table: Trips

+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| id          | int      |
| client_id   | int      |
| driver_id   | int      |
| city_id     | int      |
| status      | enum     |
| request_at  | varchar  |
+-------------+----------+
id is the primary key (column with unique values) for this table.
The table holds all taxi trips. Each trip has a unique id, while client_id and driver_id are foreign keys to the users_id at the Users table.
Status is an ENUM (category) type of ('completed', 'cancelled_by_driver', 'cancelled_by_client').

Table: Users

+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| users_id    | int      |
| banned      | enum     |
| role        | enum     |
+-------------+----------+
users_id is the primary key (column with unique values) for this table.
The table holds all users. Each user has a unique users_id, and role is an ENUM type of ('client', 'driver', 'partner').
banned is an ENUM (category) type of ('Yes', 'No').

The cancellation rate is computed by dividing the number of canceled (by client or driver) requests with unbanned users by the total number of requests with unbanned users on that day.

Write a solution to find the cancellation rate of requests with unbanned users (both client and driver must not be banned) each day between "2013-10-01" and "2013-10-03" with at least one trip. Round Cancellation Rate to two decimal points.

Return the result table in any order.

The result format is in the following example.



Example 1:

Input:
Trips table:
+----+-----------+-----------+---------+---------------------+------------+
| id | client_id | driver_id | city_id | status              | request_at |
+----+-----------+-----------+---------+---------------------+------------+
| 1  | 1         | 10        | 1       | completed           | 2013-10-01 |
| 2  | 2         | 11        | 1       | cancelled_by_driver | 2013-10-01 |
| 3  | 3         | 12        | 6       | completed           | 2013-10-01 |
| 4  | 4         | 13        | 6       | cancelled_by_client | 2013-10-01 |
| 5  | 1         | 10        | 1       | completed           | 2013-10-02 |
| 6  | 2         | 11        | 6       | completed           | 2013-10-02 |
| 7  | 3         | 12        | 6       | completed           | 2013-10-02 |
| 8  | 2         | 12        | 12      | completed           | 2013-10-03 |
| 9  | 3         | 10        | 12      | completed           | 2013-10-03 |
| 10 | 4         | 13        | 12      | cancelled_by_driver | 2013-10-03 |
+----+-----------+-----------+---------+---------------------+------------+
Users table:
+----------+--------+--------+
| users_id | banned | role   |
+----------+--------+--------+
| 1        | No     | client |
| 2        | Yes    | client |
| 3        | No     | client |
| 4        | No     | client |
| 10       | No     | driver |
| 11       | No     | driver |
| 12       | No     | driver |
| 13       | No     | driver |
+----------+--------+--------+
Output:
+------------+-------------------+
| Day        | Cancellation Rate |
+------------+-------------------+
| 2013-10-01 | 0.33              |
| 2013-10-02 | 0.00              |
| 2013-10-03 | 0.50              |
+------------+-------------------+
Explanation:
On 2013-10-01:
  - There were 4 requests in total, 2 of which were canceled.
  - However, the request with Id=2 was made by a banned client (User_Id=2), so it is ignored in the calculation.
  - Hence there are 3 unbanned requests in total, 1 of which was canceled.
  - The Cancellation Rate is (1 / 3) = 0.33
On 2013-10-02:
  - There were 3 requests in total, 0 of which were canceled.
  - The request with Id=6 was made by a banned client, so it is ignored.
  - Hence there are 2 unbanned requests in total, 0 of which were canceled.
  - The Cancellation Rate is (0 / 2) = 0.00
On 2013-10-03:
  - There were 3 requests in total, 1 of which was canceled.
  - The request with Id=8 was made by a banned client, so it is ignored.
  - Hence there are 2 unbanned request in total, 1 of which were canceled.
  - The Cancellation Rate is (1 / 2) = 0.50


# **Resposta Correta:**

In [ ]:
SELECT t.request_At as Day
    , round(
        count(case when t.status like 'cancelled%' then 1 end) / (sum(case
    when t.status in ('cancelled_by_driver', 'cancelled_by_client', 'completed') then 1 else 0 end)), 2) as 'Cancellation Rate'
FROM trips t
join users c
    on c.users_id = t.client_id and c.banned like 'No'
join users d
    on d.users_id = t.driver_id and d.banned like 'No'
where t.request_at BETWEEN date('2013-10-01') and date('2013-10-03')
group by t.request_At
order by t.request_at asc

# **Explicação:**

# 1. JOIN com a tabela Users

join users c

    on c.users_id = t.client_id and c.banned like 'No'

join users d

    on d.users_id = t.driver_id and d.banned like 'No'

Aqui, a tabela Trips é unida duas vezes com a tabela Users:


Uma vez para verificar se o cliente (client_id) não está banido.


Outra vez para verificar se o motorista (driver_id) não está banido.

Só entram na conta as viagens em que ambos são usuários não banidos.

# 2. Filtro de datas


    where t.request_at BETWEEN date('2013-10-01') and date('2013-10-03')

Limita os resultados apenas aos dias desejados.


# 3. Agrupamento por dia


    group by t.request_At

Agrupa os dados por dia (request_at) para calcular a taxa de cancelamento diária.

# 4. Cálculo da taxa de cancelamento

    round(
    count(case when t.status like 'cancelled%' then 1 end) /
    (sum(case when t.status in ('cancelled_by_driver', 'cancelled_by_client', 'completed') then 1 else 0 end)), 2) as 'Cancellation Rate'
Numerador:
    ount(case when t.status like 'cancelled%' then 1 end)

Conta quantas viagens foram canceladas (por cliente ou motorista).

Usa like 'cancelled%' para pegar ambos os tipos de cancelamento.

Denominador: sum(case when t.status in (...) then 1 else 0 end)

Conta todas as viagens válidas (completadas ou canceladas).

Ignora qualquer status fora dos três principais.

round(..., 2): Arredonda o resultado para duas casas decimais, como pedido.

# 5. Ordenação

    order by t.request_at asc
Organiza os resultados em ordem crescente de data.

